In [0]:
%sql
/****************************************************************/
/* SCHEMAS                                                      */
/****************************************************************/
CREATE SCHEMA IF NOT EXISTS `latam-md-finance`.test4;
CREATE SCHEMA IF NOT EXISTS `latam-md-finance`.control;
CREATE SCHEMA IF NOT EXISTS `latam-md-finance`.bronze;

In [0]:
#***************************************************************/
# CONTROL                                                      */
#***************************************************************/

#CREATE TABLE - Control - sys_process_source
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

schema = StructType([
    StructField("fileName", StringType(), nullable=True),
    StructField("sourceFolder", StringType(), nullable=True),
    StructField("fileExtension", StringType(), nullable=True),
    StructField("fileDelimiter", StringType(), nullable=True),
    StructField("fileNameMask", StringType(), nullable=True),
    StructField("fileEncoding", StringType(), nullable=True),
    StructField("fileCatalog", StringType(), nullable=True),
    StructField("fileSchema", StringType(), nullable=True),
    StructField("tableName", StringType(), nullable=True),
    StructField("sys_statusCode", StringType(), nullable=True),
    StructField("sys_createdByName", StringType()),
    StructField("sys_createdOn", TimestampType()),
    StructField("sys_modifiedByName", StringType()),
    StructField("sys_modifiedOn", TimestampType())
])

# create empty table
empty_df = spark.createDataFrame([], schema)
empty_df.write.format("delta").saveAsTable("`latam-md-finance`.control.sys_process_source")


In [0]:
%sql 
--Add comments & defaults
ALTER TABLE `latam-md-finance`.control.sys_process_source
SET TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

ALTER TABLE `latam-md-finance`.control.sys_process_source
ALTER COLUMN sys_createdOn SET DEFAULT getdate();

ALTER TABLE `latam-md-finance`.control.sys_process_source
ALTER COLUMN sys_modifiedOn SET DEFAULT getdate();


ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN fileName fileName STRING COMMENT 'File or process Name (Natural Key)';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN sourceFolder sourceFolder STRING COMMENT 'Source Folder name/path';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN fileExtension fileExtension STRING COMMENT 'File Extension';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN fileDelimiter fileDelimiter STRING COMMENT 'File Delimiter';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN fileNameMask fileNameMask STRING COMMENT 'File Name Mask';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN fileEncoding fileEncoding STRING COMMENT 'File Encoding';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN fileCatalog fileCatalog STRING COMMENT 'Catalog';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN fileSchema fileSchema STRING COMMENT 'Source Schema - in schema file';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN tableName tableName STRING COMMENT 'Target table name';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN sys_statusCode sys_statusCode STRING COMMENT 'System Row Status Code';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN sys_createdByName sys_createdByName STRING COMMENT 'System Created by Name';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN sys_createdOn sys_createdOn TIMESTAMP COMMENT 'System Created on';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN sys_modifiedByName sys_modifiedByName STRING COMMENT 'System Modified by Name';
ALTER TABLE `latam-md-finance`.control.sys_process_source CHANGE COLUMN sys_modifiedOn sys_modifiedOn TIMESTAMP COMMENT 'System Modified on';

In [0]:
%sql

insert into `latam-md-finance`.control.sys_process_source (fileName,sourceFolder,fileExtension,fileDelimiter,fileNameMask,fileEncoding,fileCatalog,fileSchema,tableName,sys_statusCode,sys_createdByName,sys_modifiedByName)
values('tdf_fin_variable_cost_act','raw','txt','\\t','{fileName}_{sourceSystem}_{yyyymmdd}_{hhmmss}.{fileExtension}','ISO-8859-1','latam-md-finance','fin-act-raw','tb_fin_variable_cost_act','A','vicky','vicky');

In [0]:
#***************************************************************/
# FINANCE                                                      */
#***************************************************************/

#CREATE TABLE - Bronze - tb_fin_variable_cost_act
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

schema = StructType([
    StructField("SOURCE_SYSTEM_CODE", StringType(), nullable=True),
    StructField("COMPANY_CODE", StringType(), nullable=True),
    StructField("POSTING_DATE", StringType(), nullable=True),
    StructField("POSTING_DOC_TYPE", StringType(), nullable=True),
    StructField("POSTING_DOC_NUMBER", StringType(), nullable=True),
    StructField("POSTING_DOC_LINE_NUMBER", StringType(), nullable=True),
    StructField("POSTING_TEXT", StringType(), nullable=True),
    StructField("ORIG_DOC_ENTRY_DATE", StringType(), nullable=True),
    StructField("ORIG_DOC_ISSUE_DATE", StringType(), nullable=True),
    StructField("REVERSE_POSTING_DOC_NUMBER", StringType(), nullable=True),
    StructField("COST_CENTER_CODE", StringType(), nullable=True),
    StructField("SAP_ACCOUNT_CODE", StringType(), nullable=True),
    StructField("SAP_ACCOUNT_NAME", StringType(), nullable=True),
    StructField("OFF_SAP_ACCOUNT_CODE", StringType(), nullable=True),
    StructField("OFF_SAP_ACCOUNT_NAME", StringType(), nullable=True),
    StructField("TRANS_CURRENCY_CODE", StringType(), nullable=True),
    StructField("LOCAL_CURRENCY_CODE", StringType(), nullable=True),
    StructField("USER_NAME", StringType(), nullable=True),
    StructField("EXPENSE_AMOUNT_LC", StringType(), nullable=True),
    StructField("EXPENSE_AMOUNT_TC", StringType(), nullable=True),
    StructField("TRANS_UM_CODE", StringType(), nullable=True),
    StructField("EXPENSE_QTY_TRANS_UM", StringType(), nullable=True),
    StructField("VENDOR_CODE", StringType(), nullable=True),
    StructField("VENDOR_NAME", StringType(), nullable=True),
    StructField("COST_CENTER_NAME", StringType(), nullable=True),
    StructField("CCT_TYPE_LD_CD", StringType(), nullable=True),
    StructField("DW_ROW_STAT_CD", StringType()),
    StructField("DW_ISRT_TS", TimestampType()),
    StructField("DW_UPDT_TS", TimestampType())
])

# create empty table
empty_df = spark.createDataFrame([], schema)
empty_df.write.format("delta").mode("overwrite").save('s3://latam-md-finance-bronze/tb_fin_variable_cost_act')



In [0]:
table = "`latam-md-finance`.bronze.tb_fin_variable_cost_act"
path = "s3://latam-md-finance-bronze/tb_fin_variable_cost_act"
spark.sql("CREATE TABLE IF NOT EXISTS " + table + " USING DELTA LOCATION '" +  path + "'")


In [0]:
%sql 
--Add comments & defaults
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act
SET TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act
ALTER COLUMN DW_ROW_STAT_CD SET DEFAULT 'A';

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act
ALTER COLUMN DW_ISRT_TS SET DEFAULT getdate();

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act
ALTER COLUMN DW_UPDT_TS SET DEFAULT getdate();


ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN SOURCE_SYSTEM_CODE SOURCE_SYSTEM_CODE STRING COMMENT 'Local System Code where the data have been extracted (it could be used to differentiate potential equal code with different meaning). Possible values: SAPBR, SAP1C';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN COMPANY_CODE COMPANY_CODE STRING COMMENT 'Company Code according to Prysmian Group';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_DATE POSTING_DATE STRING COMMENT 'Posting Date → Reference date (both for ETL and Reporting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_DOC_TYPE POSTING_DOC_TYPE STRING COMMENT 'Posting Document Type';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_DOC_NUMBER POSTING_DOC_NUMBER STRING COMMENT 'Reference Document Number (Posting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_DOC_LINE_NUMBER POSTING_DOC_LINE_NUMBER STRING COMMENT 'Posting Document Line Number';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_TEXT POSTING_TEXT STRING COMMENT 'Posting Text';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN ORIG_DOC_ENTRY_DATE ORIG_DOC_ENTRY_DATE STRING COMMENT 'Document Entry Date';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN ORIG_DOC_ISSUE_DATE ORIG_DOC_ISSUE_DATE STRING COMMENT 'Document Issue Date';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN REVERSE_POSTING_DOC_NUMBER REVERSE_POSTING_DOC_NUMBER STRING COMMENT 'Reverse Reference Document Number (Posting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN COST_CENTER_CODE COST_CENTER_CODE STRING COMMENT 'Cost Center Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN SAP_ACCOUNT_CODE SAP_ACCOUNT_CODE STRING COMMENT 'SAP Account Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN SAP_ACCOUNT_NAME SAP_ACCOUNT_NAME STRING COMMENT 'SAP Account Description';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN OFF_SAP_ACCOUNT_CODE OFF_SAP_ACCOUNT_CODE STRING COMMENT 'Offsetting SAP Account Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN OFF_SAP_ACCOUNT_NAME OFF_SAP_ACCOUNT_NAME STRING COMMENT 'Offsetting SAP Account Description';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN TRANS_CURRENCY_CODE TRANS_CURRENCY_CODE STRING COMMENT 'Transaction Currency Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN LOCAL_CURRENCY_CODE LOCAL_CURRENCY_CODE STRING COMMENT 'Local Currency Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN USER_NAME USER_NAME STRING COMMENT 'User Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN EXPENSE_AMOUNT_LC EXPENSE_AMOUNT_LC STRING COMMENT 'Expense in Local Currency (Value/Obj. Crcy)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN EXPENSE_AMOUNT_TC EXPENSE_AMOUNT_TC STRING COMMENT 'Expense in Transaction Currency';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN TRANS_UM_CODE TRANS_UM_CODE STRING COMMENT 'Transaction Unit of Measure Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN EXPENSE_QTY_TRANS_UM EXPENSE_QTY_TRANS_UM STRING COMMENT 'Good Receipt Quantity Stocking (Base) UoM';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN VENDOR_CODE VENDOR_CODE STRING COMMENT 'Vendor Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN VENDOR_NAME VENDOR_NAME STRING COMMENT 'Master information: Vendor Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN COST_CENTER_NAME COST_CENTER_NAME STRING COMMENT 'Cost Center Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN CCT_TYPE_LD_CD CCT_TYPE_LD_CD STRING COMMENT 'Master information: Cost Center Type Code → It indicates which ones have been included in this extract';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN DW_ROW_STAT_CD DW_ROW_STAT_CD STRING COMMENT 'Row status';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN DW_ISRT_TS DW_ISRT_TS TIMESTAMP COMMENT 'Insert Row timestamp';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN DW_UPDT_TS DW_UPDT_TS TIMESTAMP COMMENT 'Update Row timestamp';

